In [94]:
import numpy as np
import math as m
from math import sin as s
from math import cos as c
from math import acos
from math import asin
from math import atan

### REVERSE ANALYSIS FOR Cincinnati Milacron T3-776

In [6]:
### MECHANISM PARAMETERS
aL = [0,44,0,0,0] ## in inches
apL = [90,0,90,61,61] ## in degrees
S = [0,0,55,0] ## S2 through S4, in inches
## all joint angles variable

In [100]:
def fwdAnalysis(A_vec,alpha_vec,S_vec,theta_rad):
    # A_vec is a 1xN vector that contains link length (a12 through a56)
    # alpha_vec is a 1xN vector that contains twist angles (alpha12 through alpha56)
    # S_vec is a 1xN vector that contains joint offset (S2 through S5)
    # theta_rad contains the given joint angles in radians where theta_rad[0] is phi_1

    i=0
    T_F1 = np.array([m.cos(theta_rad[i]),-m.sin(theta_rad[i]),0,0,
                     m.sin(theta_rad[i]),m.cos(theta_rad[i]),0,0,
                     0,0,1,0,
                     0,0,0,1
        
    ]).reshape(4,4)

    i+=1
    j=0
    T_12 = np.array([m.cos(theta_rad[i]),-m.sin(theta_rad[i]),0,A_vec[j],
                        m.sin(theta_rad[i])*m.cos(alpha_vec[j]),m.cos(theta_rad[i])*m.cos(alpha_vec[j]),-m.sin(alpha_vec[j]),-m.sin(alpha_vec[j])*S_vec[j],
                        m.sin(theta_rad[i])*m.sin(alpha_vec[j]),m.cos(theta_rad[i])*m.sin(alpha_vec[j]),m.cos(alpha_vec[j]),m.cos(alpha_vec[j])*S_vec[j],
                        0,0,0,1
            
        ]).reshape(4,4)

    i+=1
    j+=1
    T_23 = np.array([m.cos(theta_rad[i]),-m.sin(theta_rad[i]),0,A_vec[j],
                        m.sin(theta_rad[i])*m.cos(alpha_vec[j]),m.cos(theta_rad[i])*m.cos(alpha_vec[j]),-m.sin(alpha_vec[j]),-m.sin(alpha_vec[j])*S_vec[j],
                        m.sin(theta_rad[i])*m.sin(alpha_vec[j]),m.cos(theta_rad[i])*m.sin(alpha_vec[j]),m.cos(alpha_vec[j]),m.cos(alpha_vec[j])*S_vec[j],
                        0,0,0,1
        
    ]).reshape(4,4)

    i+=1
    j+=1
    T_34 = np.array([m.cos(theta_rad[i]),-m.sin(theta_rad[i]),0,A_vec[j],
                        m.sin(theta_rad[i])*m.cos(alpha_vec[j]),m.cos(theta_rad[i])*m.cos(alpha_vec[j]),-m.sin(alpha_vec[j]),-m.sin(alpha_vec[j])*S_vec[j],
                        m.sin(theta_rad[i])*m.sin(alpha_vec[j]),m.cos(theta_rad[i])*m.sin(alpha_vec[j]),m.cos(alpha_vec[j]),m.cos(alpha_vec[j])*S_vec[j],
                        0,0,0,1
        
    ]).reshape(4,4)

    i+=1
    j+=1
    T_45 = np.array([m.cos(theta_rad[i]),-m.sin(theta_rad[i]),0,A_vec[j],
                        m.sin(theta_rad[i])*m.cos(alpha_vec[j]),m.cos(theta_rad[i])*m.cos(alpha_vec[j]),-m.sin(alpha_vec[j]),-m.sin(alpha_vec[j])*S_vec[j],
                        m.sin(theta_rad[i])*m.sin(alpha_vec[j]),m.cos(theta_rad[i])*m.sin(alpha_vec[j]),m.cos(alpha_vec[j]),m.cos(alpha_vec[j])*S_vec[j],
                        0,0,0,1
        
    ]).reshape(4,4)

    i+=1
    j+=1
    T_56 = np.array([m.cos(theta_rad[i]),-m.sin(theta_rad[i]),0,A_vec[j],
                        m.sin(theta_rad[i])*m.cos(alpha_vec[j]),m.cos(theta_rad[i])*m.cos(alpha_vec[j]),-m.sin(alpha_vec[j]),-m.sin(alpha_vec[j])*S_vec[j],
                        m.sin(theta_rad[i])*m.sin(alpha_vec[j]),m.cos(theta_rad[i])*m.sin(alpha_vec[j]),m.cos(alpha_vec[j]),m.cos(alpha_vec[j])*S_vec[j],
                        0,0,0,1
        
    ]).reshape(4,4)

    T_F6 = T_F1@T_12@T_23@T_34@T_45@T_56

    return T_F6

In [13]:
def closeTheLoop(P_6Tool,P_FTool,S_F6,a_F67):
    
    # P_6Tool = P_6Tool#.reshape(3,1)
    # P_FTool = P_FTool#.reshape(3,1)
    # S_F6 = S_F6#.reshape(3,1)
    # a_F67 = a_F67#.reshape(3,1)

    P_F6org = P_FTool - (np.dot(P_6Tool,np.array([1,0,0]))*a_F67 + np.dot(P_6Tool,np.array([0,1,0]))*np.cross(S_F6,a_F67) + np.dot(P_6Tool,np.array([0,0,1]))*S_F6)

    a_67 = 0
    alpha_67 = 90
    x_FF = np.array([1,0,0])#.reshape(3,1)

    S_F7 = np.cross(a_F67,S_F6)
    S_F1 = np.array([0,0,1])#.reshape(3,1)

    c_71 = np.dot(S_F7,S_F1)
    alpha_71 = m.acos(c_71)*(180/m.pi)
    

    ## special case 1
    if np.abs(c_71) == 1:
        S_7 = 0
        s_71 = 0

        S_1 = np.dot(-P_F6org,S_F1)

        a_71 = np.abs(-P_F6org + S_1*S_F1)

        ## special case 2
        if a_71 == 0:
            
            a_F71 = a_F67
            t_7 = 0
            a_71 = a_67
            cg_1 = np.dot(a_F71,x_FF)
            sg_1 = np.dot(np.cross(a_F71,x_FF),S_F1)
            g_1 = m.asin(sg_1)*(180/m.pi)

            return a_71,alpha_71,S_7,t_7,S_1,g_1
    
        a_F71 = (-P_F6org + S_1*S_F1)/a_71

        c7 = np.dot(a_F67,a_F71)
        s7 = np.dot(np.cross(a_F67,a_F71),S_F7)
        t_7 = m.acos(c7)*(180/m.pi)

        cg_1 = np.dot(a_F71,x_FF)
        sg_1 = np.dot(np.cross(a_F71,x_FF),S_F1)
        g_1 = m.asin(sg_1)*(180/m.pi)

        return a_71,alpha_71,S_7,t_7,S_1,g_1

    a_F71 = np.cross(S_F7,S_F1)/np.linalg.norm(np.cross(S_F7,S_F1))

    s_71 = np.dot((np.cross(S_F7,S_F1)),a_F71)

    c7 = np.dot(a_F67,a_F71)
    s7 = np.dot(np.cross(a_F67,a_F71),S_F7)
    t_7 = m.acos(c7)*(180/m.pi)
    

    cg_1 = np.dot(a_F71,x_FF)
    sg_1 = np.dot(np.cross(a_F71,x_FF),S_F1)
    g_1 = m.asin(sg_1)*(180/m.pi)


    S_7 = np.dot((np.cross(S_F1,P_F6org)),a_F71)/s_71
              
    a_71 = np.dot(np.cross(P_F6org,S_F1),S_F7)/s_71
    


    S_1 = np.dot(np.cross(P_F6org,S_F7),a_F71)/s_71
       
    return a_67, alpha_67, a_71,alpha_71,S_7,t_7,S_1,g_1

In [107]:
### REVERSE ANALYSIS CODE -- CINCINNATI MILACRON T3-776 ROBOT (RRRRRRR)
def reverseAnalysis(aL,apL,S,S6,P6tool,PFtool,SF6,aF67):
    ## Free choices, Tool Params, CTL
    a67, ap67, a71,ap71,S7,t7,S1,g1 = closeTheLoop(P6tool,PFtool,SF6,aF67)

    t7 = t7*(m.pi/180)

    a12, a23, a34, a45, a56 = aL
    ap12, ap23, ap34, ap45, ap56 = apL
    S2,S3,S4,S5 = S

    ## ANGLE TREE BRANCHES
    b1 = [] ## 7, 1a, 3a, 2a, 5a, 4a
    b2 = [] ## 7, 1a, 3a, 2a, 5b, 4b
    b3 = [] ## 7, 1a, 3b, 2b, 5c, 4c
    b4 = [] ## 7, 1a, 3b, 2b, 5d, 4d
    b5 = [] ## 7, 2a, 3c, 2c, 5e, 4e
    b6 = [] ## 7, 2a, 3c, 2c, 5f, 4f
    b7 = [] ## 7, 2a, 3d, 2d, 5g, 4g
    b8 = [] ## 7, 2a, 3d, 2d, 5h, 4h
    tree = [b1,b2,b3,b4,b5,b6,b7,b8]

    for i in tree:
        i.append(t7)


    ## THETA 1 (2 UNIQUE SOLUTIONS)
    t1t = atan(((S6 * c(t7) * c(ap71)) + (S7 * s(ap71)) ) / ( (S6 * s(t7)) + a71 ))

    t1 = [t1t,t1t+m.pi]
    for i,j in enumerate(tree):
        if i>3:
            k=1
            j.append(t1[1])
        else:
            k=0
            j.append(t1[0])
    p1 = [(t1[0] - g1),(t1[1]-g1)]

    ## THETA 3 (4 UNIQUE SOLUTIONS) [DOUBLED]

    
    A3 = a23 * c(t2) + S4 * s(t2+t3)
    B3 = a23 * s(t2) - S4 * c(t2+t3)

    t3 = (A3**2 + B3**2 - a23**2 - S4**2) / (2 * a23 * S4)

    # ## THETA 2 (4 UNIQUE SOLUTIONS)
    
    # t2 = 1

    # ## THETA 5 (8 UNIQUE SOLUTIONS) [DOUBLED]
    # for i in range(len(t2)): 
    #     X7 = s(ap67) * s(t7)
    #     Y7 = -( (s(ap71) * c(ap67)) + (c(ap71) * s(ap67) * c(t7)) )
    #     Z7 = ( c(ap71) * c(ap67) ) - ( s(ap71) * s(ap67) * c(t7) )
    #     X71 = (X7 * c(t1)) + (Y7 * s(t1))
    #     Y71 = c(ap12) * ((X7 * s(t1)) + (Y7 * c(t1))) - (s(ap12) * Z7)
    #     Z71 = s(ap12) * ((X7 * s(t1)) + (Y7 * c(t1))) + (c(ap12) * Z7)
    #     X712 = (X71 * c(t3)) - (Y71 * s(t3))
    #     Y712 = c(ap23) * ((X71 * s(t3)) + (Y71 * c(t3))) - (s(ap23) * Z71)
    #     Z712 = s(ap23) * ((X71 * s(t3)) + (Y71 * c(t3))) + (c(ap23) * Z71)
    #     Z7123 = s(ap34) * ((X712 * s(t4)) + (Y712 * c(t4))) + (c(ap34) * Z712)

    #     t5 = acos( (c(ap56) * c(ap56) - Z7123) / (s(ap56) * s(ap45)) )

    # ## THETA 4 (8 UNIQUE SOLUTIONS)


    # A = np.array([[Xbar5, -Ybar5],[Ybar5, Xbar5]

    # ])
    
    # b = np.array([X7123,-Y7123

    # ])

    # t4c, t4s = np.linalg.solve(A,b)

    # t4c = acos(t4c)
    # t4s = asin(t4s)

    # t4 = 1

    # ## THETA 6 (8 UNIQUE SOLUTIONS)
    # X4 =
    # Y4 =
    # Z4 =
    # X43 =
    # Y43 =
    # Z43 =
    # X432 =
    # Y432 =
    # Z432 =
    # X4321 =
    # Y4321 =
    # Z4321 =
    # X43217 = 
    # Y43217 = 

    # t6c = acos( X43217 / s(ap56) )
    # t6s = asin( Y43217 / s(ap56) )

    # t6 = 1







    return tree

In [110]:
## UFID is 57758617 -> Check answer on forward analysis for Cincinnati Milacron T3-776

## 
P6tool = np.array([5,3,7])
PFtool = np.array([25,23,24])
SF6 = np.array([0.177,0.884,-0.433])
aF67 = np.array([-0.153,0.459,0.875])


# link length in INCHES
aL = np.array([0,44,0,0,0

])

# twist angle in DEGREES
apL = np.array([90,0,270,90,90
    
])*(m.pi/180) ## convert to radians ##


# joint offest in INCHES
S6 = 6
S = np.array([0,0,55,0
              
]) # PICK S6

# joint angle in RADIANS... 
angles = reverseAnalysis(aL,apL,S,S6,P6tool,PFtool,SF6,aF67)


#Milacron_t3 = fwdAnalysis(aL,apL,S,angles)
print(angles)
#print(Milacron_t3.round(3))

[[1.1116137109896878, -1.0096406194686314], [1.1116137109896878, -1.0096406194686314], [1.1116137109896878, -1.0096406194686314], [1.1116137109896878, -1.0096406194686314], [1.1116137109896878, 2.131952034121162], [1.1116137109896878, 2.131952034121162], [1.1116137109896878, 2.131952034121162], [1.1116137109896878, 2.131952034121162]]


In [42]:
test = [[1],[1,2],[1,2,3,4],[1,2,3,4,5,6,7,8]]
j=0
for i in test:
    if len(i) == 1:
        
    j+=1

[1, 2]
[1, 2, 3, 4]
[1, 2, 3, 4, 5, 6, 7, 8]


In [71]:
b1 = [] ## 7, 1a, 3a, 2a, 5a, 4a
b2 = [] ## 7, 1a, 3a, 2a, 5b, 4b
b3 = [] ## 7, 1a, 3b, 2b, 5c, 4c
b4 = [] ## 7, 1a, 3b, 2b, 5d, 4d
b5 = [] ## 7, 2a, 3c, 2c, 5e, 4e
b6 = [] ## 7, 2a, 3c, 2c, 5f, 4f
b7 = [] ## 7, 2a, 3d, 2d, 5g, 4g
b8 = [] ## 7, 2a, 3d, 2d, 5h, 4h
tree = [b1,b2,b3,b4,b5,b6,b7,b8]

t7 = 10
for i in tree:
    i.append(t7)
t1 = [5,2]
for i,j in enumerate(tree):
    if i>3:
        j.append(t1[1])
    else:
        j.append(t1[0])

t2 = [3,6,9,12]
for i,j in enumerate(tree):
    if i>1 and i<4:
        j.append(t2[1])
    if i>3 and i<6:
        j.append(t2[2])
    if i>5:
        j.append(t2[3])
    if i<=1:
        j.append(t2[0])

t3 = [5,25,125,625] 
for i,j in enumerate(tree):
    if i>1 and i<4:
        j.append(t3[1])
    if i>3 and i<6:
        j.append(t3[2])
    if i>5:
        j.append(t3[3])
    if i<=1:
        j.append(t3[0])

t4 = [7,14,28,56,112,224,448,896]
for i,j in enumerate(tree):
    j.append(t4[i])

t5 = [1,2,3,4,5,6,7,8]
for i,j in enumerate(tree):
    j.append(t5[i])

print(tree)

[[10, 5, 3, 5, 7, 1], [10, 5, 3, 5, 14, 2], [10, 5, 6, 25, 28, 3], [10, 5, 6, 25, 56, 4], [10, 2, 9, 125, 112, 5], [10, 2, 9, 125, 224, 6], [10, 2, 12, 625, 448, 7], [10, 2, 12, 625, 896, 8]]


In [93]:
m.atan(m.tan(67*(m.pi/180)))*(180/m.pi),m.atan(m.tan((180+67)*(m.pi/180)))*(180/m.pi)

(67.0, 66.99999999999999)